In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Load data from Excel (replace with the correct file path)
df = pd.read_excel('../data.xlsx', sheet_name='Rekapan')

In [ ]:
# Display the first and last few rows of the dataset
print(df.head())
print(df.tail())

In [ ]:
# Display basic descriptive statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Visualize data distribution
df.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
# Filter relevant columns
data_filtered = df[['Tanggal', 'SO', 'TERKIRIM', 'Harga Komoditas Bijih Besi', 'Indeks Produksi Dalam Negeri', 'Data Inflasi', 'Kurs']]

# Rename columns for easier handling
data_filtered.columns = ['Tanggal', 'SO', 'Terkirim', 'Harga Komoditas', 'Indeks Produksi', 'Data Inflasi', 'Kurs']

# Convert 'Tanggal' column to datetime
data_filtered['Tanggal'] = pd.to_datetime(data_filtered['Tanggal'])

# Convert 'Indeks Produksi' to numeric (coercing errors)
data_filtered['Indeks Produksi'] = pd.to_numeric(data_filtered['Indeks Produksi'], errors='coerce')

# Create 'bulan_tahun' column to group by month and year
data_filtered['bulan_tahun'] = data_filtered['Tanggal'].dt.to_period('M')

# Monthly Summary by 'bulan_tahun'
monthly_summary = data_filtered.groupby('bulan_tahun').agg({
    'SO': 'sum',
    # Uncomment other columns as needed
    # 'Terkirim': 'sum',
    # 'Harga Komoditas': 'mean',
    # 'Indeks Produksi': 'mean',
    # 'Data Inflasi': 'mean',
    # 'Kurs': 'mean'
}).reset_index()

# Display monthly summary
print(monthly_summary)

In [ ]:
# Set alpha value for Double Exponential Smoothing (DES)
alpha = 0.1

# Initialize columns for DES and error metrics
monthly_summary['Single'] = np.nan
monthly_summary['Double'] = np.nan
monthly_summary['At'] = np.nan
monthly_summary['Bt'] = np.nan
monthly_summary['DES Forecast'] = np.nan
monthly_summary['Error'] = np.nan
monthly_summary['MAD'] = np.nan
monthly_summary['MSE'] = np.nan
monthly_summary['MAPE'] = np.nan

# Set initial Single, Double, At, and Bt values based on the first observation
monthly_summary.loc[0, 'Single'] = monthly_summary.loc[0, 'SO']
monthly_summary.loc[0, 'Double'] = monthly_summary.loc[0, 'SO']
monthly_summary.loc[0, 'At'] = (2 * monthly_summary.loc[0, 'Single']) - monthly_summary.loc[0, 'Double']
monthly_summary.loc[0, 'Bt'] = 0  # Set initial trend (Bt) to 0

# Apply DES formula and calculate error metrics for each subsequent row
for i in range(1, len(monthly_summary)):
    # Single Exponential smoothing (S't)
    monthly_summary.loc[i, 'Single'] = (alpha * monthly_summary.loc[i, 'SO']) + ((1 - alpha) * monthly_summary.loc[i-1, 'Single'])
    
    # Double Exponential smoothing (S''t)
    monthly_summary.loc[i, 'Double'] = (alpha * monthly_summary.loc[i, 'Single']) + ((1 - alpha) * monthly_summary.loc[i-1, 'Double'])
    
    # At (level)
    monthly_summary.loc[i, 'At'] = (2 * monthly_summary.loc[i, 'Single']) - monthly_summary.loc[i, 'Double']
    
    # Bt (trend)
    monthly_summary.loc[i, 'Bt'] = (alpha / (1 - alpha)) * (monthly_summary.loc[i, 'Single'] - monthly_summary.loc[i, 'Double'])
    
    # DES Forecast for the next period
    monthly_summary.loc[i, 'DES Forecast'] = monthly_summary.loc[i-1, 'At'] + monthly_summary.loc[i-1, 'Bt']
    
    # Error (actual - forecast)
    monthly_summary.loc[i, 'Error'] = monthly_summary.loc[i, 'SO'] - monthly_summary.loc[i, 'DES Forecast']
    
    # MAD (Mean Absolute Deviation)
    monthly_summary.loc[i, 'MAD'] = abs(monthly_summary.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    monthly_summary.loc[i, 'MSE'] = monthly_summary.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if monthly_summary.loc[i, 'SO'] != 0:
        monthly_summary.loc[i, 'MAPE'] = (abs(monthly_summary.loc[i, 'Error']) / monthly_summary.loc[i, 'SO']) * 100
    else:
        monthly_summary.loc[i, 'MAPE'] = np.nan

# Show the result with DES forecast and error metrics
print(monthly_summary)

In [13]:
# Forecast for future months (12 months ahead)
months_to_forecast = 12

# Get the last month from the original data
last_month = monthly_summary['bulan_tahun'].max().to_timestamp()

# Generate dummy months starting from the next month after the last month in the original data
dummy_data = pd.DataFrame({
    'bulan_tahun': pd.date_range(start=last_month + pd.offsets.MonthBegin(1), periods=months_to_forecast, freq='M').to_period('M'),
    'SO': np.zeros(months_to_forecast)  # Set SO to 0 for future months
})

# Combine original data with the dummy future data
extended_data = pd.concat([monthly_summary[['bulan_tahun', 'SO']], dummy_data], ignore_index=True)

# Initialize columns for DES forecast in the extended data
extended_data['Single'] = np.nan
extended_data['Double'] = np.nan
extended_data['At'] = np.nan
extended_data['Bt'] = np.nan
extended_data['DES Forecast'] = np.nan
extended_data['Error'] = np.nan
extended_data['MAD'] = np.nan
extended_data['MSE'] = np.nan
extended_data['MAPE'] = np.nan

# Set initial values for the DES components based on the first row of the data
extended_data.loc[0, 'Single'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'Double'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'At'] = (2 * extended_data.loc[0, 'Single']) - extended_data.loc[0, 'Double']
extended_data.loc[0, 'Bt'] = 0  # Assume no initial trend

# Apply DES formula to the entire dataset (historical and future periods)
for i in range(1, len(extended_data)):
    # Apply DES formula for all periods (historical and future)
    extended_data.loc[i, 'Single'] = (alpha * extended_data.loc[i, 'SO']) + ((1 - alpha) * extended_data.loc[i-1, 'Single'])
    extended_data.loc[i, 'Double'] = (alpha * extended_data.loc[i, 'Single']) + ((1 - alpha) * extended_data.loc[i-1, 'Double'])
    extended_data.loc[i, 'At'] = (2 * extended_data.loc[i, 'Single']) - extended_data.loc[i, 'Double']
    extended_data.loc[i, 'Bt'] = (alpha / (1 - alpha)) * (extended_data.loc[i, 'Single'] - extended_data.loc[i, 'Double'])
    extended_data.loc[i, 'DES Forecast'] = extended_data.loc[i-1, 'At'] + extended_data.loc[i-1, 'Bt']
    
    # Error
    extended_data.loc[i, 'Error'] = extended_data.loc[i, 'SO'] - extended_data.loc[i, 'DES Forecast']
    
    # MAD (Mean Absolute Deviation)
    extended_data.loc[i, 'MAD'] = abs(extended_data.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    extended_data.loc[i, 'MSE'] = extended_data.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if extended_data.loc[i, 'SO'] != 0:
        extended_data.loc[i, 'MAPE'] = (abs(extended_data.loc[i, 'Error']) / extended_data.loc[i, 'SO']) * 100
    else:
        extended_data.loc[i, 'MAPE'] = np.nan

# Display the extended data with DES forecast results
print(extended_data)


   bulan_tahun         SO        Single        Double            At  \
0      2018-01  2845126.0  2.845126e+06  2.845126e+06  2.845126e+06   
1      2018-02  2751060.0  2.835719e+06  2.844185e+06  2.827253e+06   
2      2018-03  3751325.0  2.927280e+06  2.852495e+06  3.002065e+06   
3      2018-04  3350350.0  2.969587e+06  2.864204e+06  3.074970e+06   
4      2018-05  2733224.0  2.945951e+06  2.872379e+06  3.019523e+06   
..         ...        ...           ...           ...           ...   
83     2024-12        0.0  1.136657e+06  2.132202e+06  1.411115e+05   
84     2025-01        0.0  1.022991e+06  2.021281e+06  2.470128e+04   
85     2025-02        0.0  9.206919e+05  1.911222e+06 -6.983804e+04   
86     2025-03        0.0  8.286227e+05  1.802962e+06 -1.457165e+05   
87     2025-04        0.0  7.457605e+05  1.697242e+06 -2.057209e+05   

               Bt  DES Forecast          Error            MAD           MSE  \
0        0.000000           NaN            NaN            NaN       

C:\Users\chris\AppData\Local\Temp\ipykernel_416\3117380225.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'bulan_tahun': pd.date_range(start=last_month + pd.offsets.MonthBegin(1), periods=months_to_forecast, freq='M').to_period('M'),


In [ ]:
# Plot Actual SO vs DES Forecast, including future months
plt.figure(figsize=(14, 6))
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['SO'], label='Actual SO', marker='o')
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['DES Forecast'], label='DES Forecast', marker='x')
plt.xlabel('Month-Year')
plt.ylabel('SO')
plt.xticks(rotation=45)
plt.title('Actual SO vs DES Forecast (Including Future Months)')
plt.legend()
plt.tight_layout()
plt.show()